<a href="https://colab.research.google.com/github/platypus2000jp/Colaboratory-R-mode/blob/main/%E4%BE%8B%E9%A1%8C_%E9%87%8E%E9%B3%A5%E3%81%AE%E6%A8%99%E8%AD%98%E8%AA%BF%E6%9F%BB%E3%81%AE%E3%83%99%E3%82%A4%E3%82%BA%E3%83%A2%E3%83%87%E3%83%AB_RStan%E3%82%92%E4%BD%BF%E3%81%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

野鳥の標識調査について機械学習またはベイズのモデルの解析例を示してください。Rでお願いします。rstanを使ってください。

ChatGPT:
ChatGPT
野鳥の標識調査（標識再捕法）は、野鳥個体群の動態や生存率、移動パターンを調査するための手法です。ベイズ統計や機械学習のアプローチを使用することで、個体群のパラメータ推定や予測精度を高めることができます。ここでは、rstanを使ったベイズ推定の例として、簡単な「標識再捕法」に基づく生存率モデルをRで構築します。

データ構造の説明
標識再捕データには、以下のような情報が含まれます。

N: 放鳥または標識された個体数
y: その後の捕獲履歴 (例: 0は再捕獲されなかった、1は再捕獲された)
T: 調査期間
例えば、1期目に100羽の野鳥に標識を付け、その後の3年間で再捕率を記録するようなデータを想定します。

ベイズモデルの基本的な構造
ここでは、φを生存率、pを捕獲確率としてモデル化します。生存率は調査期間中に個体が生き残る確率であり、捕獲確率は捕獲される確率です。これらのパラメータをベイズ推定します。

RStanを使ったコード例
Stanコードの記述: Stanコードでは、生存率phiと捕獲確率pを推定します。これらはベータ分布の事前分布を持ちます。
r
コードをコピーする
library(rstan)

# Stanコードの作成
stan_code <- "
data {
  int<lower=0> N; // 標識個体数
  int<lower=0> T; // 調査期間
  int<lower=0,upper=1> y[N, T]; // 捕獲履歴
}
parameters {
  real<lower=0,upper=1> phi; // 生存率
  real<lower=0,upper=1> p;   // 捕獲確率
}
model {
  phi ~ beta(1, 1); // 生存率の事前分布
  p ~ beta(1, 1);   // 捕獲確率の事前分布

  for (i in 1:N) {
    for (t in 2:T) {
      // キャプチャ履歴の尤度
      if (y[i, t-1] == 1) {
        y[i, t] ~ bernoulli(phi * p);
      }
    }
  }
}
"

# Stanコードをファイルに保存しても良いが、今回は直接使います。

# 2. データの準備
set.seed(123)
N <- 100  # 標識された個体数
T <- 5    # 調査期間
phi_true <- 0.7  # 真の生存率
p_true <- 0.5    # 真の捕獲確率

# 捕獲履歴のシミュレーション
y <- matrix(0, nrow = N, ncol = T)
for (i in 1:N) {
  y[i, 1] <- rbinom(1, 1, p_true)  # 初年度の捕獲
  for (t in 2:T) {
    if (y[i, t-1] == 1) {  # 前の年に捕獲されていたら
      survived <- rbinom(1, 1, phi_true)
      y[i, t] <- ifelse(survived == 1, rbinom(1, 1, p_true), 0)
    }
  }
}

# 3. Stanにデータを渡してモデルを実行
stan_data <- list(N = N, T = T, y = y)
fit <- stan(model_code = stan_code, data = stan_data, iter = 2000, chains = 4)

# 4. 結果の確認
print(fit, pars = c("phi", "p"))
コードの解説
Stanモデルでは、生存率phiと捕獲確率pをそれぞれベータ分布から事前に設定しています。調査期間中のデータ（捕獲履歴y）を用いてこれらのパラメータを推定します。
yは個体ごとの再捕獲データです。初年度に標識された個体が、次年度以降に生存し再捕獲されたかどうかが0または1で記録されています。
結果の解釈
fitの結果から推定されたphi（生存率）とp（捕獲確率）が確認できます。真の値（phi_true = 0.7、p_true = 0.5）と比較することでモデルの精度を評価します。

参考
ベイズモデルの詳細やrstanの使い方については、以下のドキュメントが役立ちます。

Stan公式ドキュメント
RStanのチュートリアル
このアプローチを拡張すれば、より複雑な個体群動態や環境要因を考慮したモデルも構築できます。

In [ ]:
##rstanがインストールされていないならインストールする
##2021年1月現在うまく動く
x<-installed.packages()
if(any(x[,1]== 'rstan') == FALSE){
    system("add-apt-repository -y ppa:marutter/rrutter4.0")
    system("add-apt-repository -y ppa:c2d4u.team/c2d4u4.0+")
    system("add-apt-repository ppa:cran/v8")
    system("apt-get update")
    system("apt-get install libnode-dev")
    system("apt-get install r-cran-rstan")
}

In [ ]:
# パッケージの読み込み
library(rstan)

Loading required package: StanHeaders


rstan version 2.32.6 (Stan version 2.32.2)


For execution on a local, multicore CPU with excess RAM we recommend calling
options(mc.cores = parallel::detectCores()).
To avoid recompilation of unchanged Stan programs, we recommend calling
rstan_options(auto_write = TRUE)
For within-chain threading using `reduce_sum()` or `map_rect()` Stan functions,
change `threads_per_chain` option:
rstan_options(threads_per_chain = 1)




In [ ]:
# Stanコードの作成
stan_code <- "
data {
  int<lower=0> N; // 標識個体数
  int<lower=0> T; // 調査期間
  int<lower=0,upper=1> y[N, T]; // 捕獲履歴
}
parameters {
  real<lower=0,upper=1> phi; // 生存率
  real<lower=0,upper=1> p;   // 捕獲確率
}
model {
  phi ~ beta(1, 1); // 生存率の事前分布
  p ~ beta(1, 1);   // 捕獲確率の事前分布

  for (i in 1:N) {
    for (t in 2:T) {
      // キャプチャ履歴の尤度
      if (y[i, t-1] == 1) {
        y[i, t] ~ bernoulli(phi * p);
      }
    }
  }
}
"

# Stanコードをファイルに保存しても良いが、今回は直接使います。

In [ ]:
# 2. データの準備
set.seed(123)
N <- 100  # 標識された個体数
T <- 5    # 調査期間
phi_true <- 0.7  # 真の生存率
p_true <- 0.5    # 真の捕獲確率

In [ ]:
# 捕獲履歴のシミュレーション
y <- matrix(0, nrow = N, ncol = T)
for (i in 1:N) {
  y[i, 1] <- rbinom(1, 1, p_true)  # 初年度の捕獲
  for (t in 2:T) {
    if (y[i, t-1] == 1) {  # 前の年に捕獲されていたら
      survived <- rbinom(1, 1, phi_true)
      y[i, t] <- ifelse(survived == 1, rbinom(1, 1, p_true), 0)
    }
  }
}


In [ ]:
# 3. Stanにデータを渡してモデルを実行
stan_data <- list(N = N, T = T, y = y)
fit <- stan(model_code = stan_code, data = stan_data, iter = 2000, chains = 4)


SAMPLING FOR MODEL 'anon_model' NOW (CHAIN 1).
Chain 1: 
Chain 1: Gradient evaluation took 4.7e-05 seconds
Chain 1: 1000 transitions using 10 leapfrog steps per transition would take 0.47 seconds.
Chain 1: Adjust your expectations accordingly!
Chain 1: 
Chain 1: 
Chain 1: Iteration:    1 / 2000 [  0%]  (Warmup)
Chain 1: Iteration:  200 / 2000 [ 10%]  (Warmup)
Chain 1: Iteration:  400 / 2000 [ 20%]  (Warmup)
Chain 1: Iteration:  600 / 2000 [ 30%]  (Warmup)
Chain 1: Iteration:  800 / 2000 [ 40%]  (Warmup)
Chain 1: Iteration: 1000 / 2000 [ 50%]  (Warmup)
Chain 1: Iteration: 1001 / 2000 [ 50%]  (Sampling)
Chain 1: Iteration: 1200 / 2000 [ 60%]  (Sampling)
Chain 1: Iteration: 1400 / 2000 [ 70%]  (Sampling)
Chain 1: Iteration: 1600 / 2000 [ 80%]  (Sampling)
Chain 1: Iteration: 1800 / 2000 [ 90%]  (Sampling)
Chain 1: Iteration: 2000 / 2000 [100%]  (Sampling)
Chain 1: 
Chain 1:  Elapsed Time: 0.247 seconds (Warm-up)
Chain 1:                0.251 seconds (Sampling)
Chain 1:                0.49

In [ ]:
# 4. 結果の確認
print(fit, pars = c("phi", "p"))

Inference for Stan model: anon_model.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

    mean se_mean   sd 2.5%  25%  50%  75% 97.5% n_eff Rhat
phi 0.60    0.01 0.19 0.31 0.45 0.58 0.75  0.97  1020 1.01
p   0.63    0.01 0.19 0.32 0.47 0.61 0.78  0.97   928 1.01

Samples were drawn using NUTS(diag_e) at Thu Oct 24 04:47:33 2024.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
convergence, Rhat=1).
